# Meta-analysis 2: AFR Only
- **Project:** GP2 AFR-AAC meta-GWAS 
- **Version:** Python/3.9
- **Status:** COMPLETE
- **Started:** 22-FEB-2023
- **Last Updated:** 22-FEB-2023
    - **Update Description:**  Notebook started

## Notebook Overview
- Meta-GWAS #2: Looking at African individuals only

### CHANGELOG
- 22-FEB-2023: Notebook started 

---
# Data Overview 

| ANCESTRY |     DATASET     | CASES | CONTROLS |  TOTAL  |           ARRAY           |                NOTES                |
|:--------:|:---------------:|:-----:|:--------:|:-------------------------:|:---------------------------------------------------------------------------------------------------------------:|:-----------------------------------:|
|    AFR   | IPDGC – Nigeria |  304  |    285   |   589   |         NeuroChip         | . | 
|    AFR   |  GP2  |  711  |   1,011  |  1,722  |        NeuroBooster       | . |
|    AAC   |  GP2 |  185  |   1,149  |  1,334  |        NeuroBooster       | . | 
|    AAC   |     23andMe     |  288  |  193,985 | 194,273 | Omni Express & GSA & 550k |        Just summary statistics       |

# Getting Started

## Importing packages

In [3]:
## Import the necessary packages 
import os
import numpy as np
import pandas as pd
import math
import numbers
import sys
import subprocess
import statsmodels.api as sm
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

## Print out package versions
## Getting packages loaded into this notebook and their versions to allow for reproducibility
    # Repurposed code from stackoverflow here: https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook

## Import packages 
import pkg_resources
import types
from datetime import date
today = date.today()
date = today.strftime("%d-%b-%Y").upper()

## Define function 
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different imported names vs. system/pip names
        # Unfortunately, there is no systematic way to get pip names from a package's imported name. You'll have to add exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

## Get a list of packages imported 
imports = list(set(get_imports()))

# The only way I found to get the version of the root package from only the name of the package is to cross-check the names of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

## Print out packages and versions 
print(f"PACKAGE VERSIONS ({date})")
for r in requirements:
    print("\t{}=={}".format(*r))

PACKAGE VERSIONS (22-FEB-2023)
	matplotlib==3.5.2
	numpy==1.22.4
	scipy==1.8.1
	pandas==1.4.3
	statsmodels==0.13.2
	seaborn==0.11.2


---
# Write out METAL command (here for documentation -- needs to be run interactively!)

```bash
module load metal 
metal << EOT 

# UNCOMMENT THE NEXT LINE TO ENABLE GenomicControl CORRECTION
SCHEME STDERR
GENOMICCONTROL ON

# === DESCRIBE AND PROCESS THE FIRST INPUT FILE ===
MARKER markerID
ALLELE effect_allele other_allele
# FREQ   maf
EFFECT beta
STDERR LOG(OR)_SE
PVALUE P
# WEIGHT OBS_CT 
PROCESS /data/GP2/projects/2023_02_MBM_AFR_AAC_GWAS/data/AFR/GP2-v4-AFR-wNIGERIAN-NB/GP2-v4-AFR-wNIGERIAN-NB-GWAS-MAF005-FEB2023.UpdatedforMETAL.tab

# === DESCRIBE AND PROCESS THE SECOND INPUT FILE ===
MARKER markerID
ALLELE effect_allele other_allele
# FREQ   maf
EFFECT beta
STDERR LOG(OR)_SE
PVALUE P
# WEIGHT OBS_CT 
PROCESS /data/GP2/projects/2023_02_MBM_AFR_AAC_GWAS/data/AFR/NIGERIAN-NC/NIGERIAN-NEUROCHIP-AFR-GWAS-MAF005-FEB2023.UpdatedforMETAL.tab

OUTFILE /data/GP2/projects/2023_02_MBM_AFR_AAC_GWAS/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL .tbl
ANALYZE HETEROGENEITY
QUIT
``` 
**Then Control+D to submit job!**

# Investigate Top Hits

In [4]:
%%bash

cat ${WORK_DIR}/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL1.tbl | awk '$6 <= 0.00000005'  | wc -l
head -1 ${WORK_DIR}/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL1.tbl 
cat ${WORK_DIR}/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL1.tbl | awk '$6 <= 0.00000005' 

2
MarkerName	Allele1	Allele2	Effect	StdErr	P-value	Direction	HetISq	HetChiSq	HetDf	HetPVal
chr1:155235878:G:T	t	g	-0.4579	0.0775	3.436e-09	--	20.9	1.264	1	0.2609
chr1:155160272:C:T	t	c	0.4178	0.0761	3.97e-08	++	32.8	1.488	1	0.2225


# Grab imputation scores for top hits

In [6]:
%%bash
cat ${WORK_DIR}/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL1.tbl | awk '$6 <= 0.00000005' | awk '{print $1}' > ${WORK_DIR}/data/AFR-META/top-AFR-ONLY-META-hits.txt

grep -E -f ${WORK_DIR}/data/AFR-META/top-AFR-ONLY-META-hits.txt ${WORK_DIR}/data/AFR/GP2-v4-AFR-wNIGERIAN-NB/GP2-v4-AFR-wNIGERIAN-NB-noSEXPHENO.pvar > ${WORK_DIR}/data/AFR-META/top-AFR-ONLY-META-hits_extractPVAR.txt

grep "#CHROM" ${WORK_DIR}/data/AFR/GP2-v4-AFR-wNIGERIAN-NB/GP2-v4-AFR-wNIGERIAN-NB-noSEXPHENO.pvar
cat ${WORK_DIR}/data/AFR-META/top-AFR-ONLY-META-hits_extractPVAR.txt

#CHROM	POS	ID	REF	ALT	FILTER	INFO
1	155160272	chr1:155160272:C:T	C	T	PASS	AF=0.25913;MAF=0.25913;R2=0.98373;IMPUTED
1	155235878	chr1:155235878:G:T	G	T	PASS	AF=0.74023;MAF=0.25977;R2=0.97060;IMPUTED


In [7]:
%%bash
head -1 ${WORK_DIR}/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL1.tbl > ${WORK_DIR}/data/AFR-META/top-AFR-ONLY-META-hits_allINFO.txt
cat ${WORK_DIR}/data/AFR-META/AFR-ONLY-META-UpdatedforMETAL1.tbl | awk '$6 <= 0.00000005' >> ${WORK_DIR}/data/AFR-META/top-AFR-ONLY-META-hits_allINFO.txt